# Data Preprocessing : Credit Card Routing for Online Purchase via Predictive Modelling

## Overview
**Pandas dataframe** is used for transforming & selecting features through python.
Below is the snapshot of the python code & explanation of steps used to derive the cumulative time as an additional feature.

The first column tmsp i.e. Original Timestamp provided in the dataset is transformed in a time difference where it’s assumed that the first transaction was initiated at 0 seconds & the next occurred 6 seconds after with the 3rd one occurring 92 seconds after the second & 98 seconds after the 1st transaction. A pattern is followed to consider the time difference obtained as one of the derived features to perform data analysis. Data is prepared using the given excel spreadsheet consisting below details:
1.	Total transactions(rows): **50470**
2.	Total features(columns): **7** (Sr. No, timestamp, country, amount, success, PSP, 3D_secured, card)

### Performing transformation of timestamp into numeric values of time difference
Below code is used to read the provided csv file & store its data into a dataframe called train_data

**pd.read_csv**=Used to read a given csv file
**pd.DataFrame**=Used to convert a given csv file into dataframe

In [ ]:
#Importing necessary libraries
import pandas as pd
import csv  
import datetime as dt
#Initiating CSV file
csv_file=pd.read_csv("C:\\MISC\\IU_downloads\\PSP_Jan_Feb_2019.csv")
#Initiating the dataframe
train_data=pd.DataFrame(csv_file,index=None);

### Writing the time difference obtained from the given timestamps in another file
This is done by opening the file through **csv writer** & iterating through all the rows thereby selecting the timestamp for each row
The timestamps of each subsequent rows are subtracted form one another to obtain a difference in **seconds**. This difference is finally written back to the csv file

**datetime.strptime**= Used to convert timestamp in required format
**csvwriter.writerow**=Used to write the obtained values one by one in each row of csv file

In [ ]:
#Providing the timestamp format for conversion of dates
format = '%Y-%m-%d %H:%M:%S'
#Writing numeric values obtained after converting dates into another csv file
with open("C:\\MISC\\IU_downloads\\PSP_Jan_Feb_2019_v1.csv","w") as csvfile:
    csvwriter=csv.writer(csvfile)
    for i in range(1,len(train_data.axes[0])):
          #Converting given timestamp into time difference of seconds starting from 0 seconds with the first record
          sec=str((dt.datetime.strptime(train_data["tmsp"][i], format)-dt.datetime.strptime(train_data["tmsp"][i-1], format)).total_seconds())
          csvwriter.writerow(sec)

## Obtaining transaction fees through python for each transaction
This is similar to a vlookup in excel file where both the files can be merged to obtain the transactional fees for every row. In Python it can be done through dataframe as below
**pd.merge**= command used to merge two files based on a common column which is "PSP" in our case, 
**pd.concat**= command used to concat two files having same columns

In [ ]:
#Initializing paths from local for provided dataset & transaction fees file
df_original=pd.DataFrame(pd.read_csv("C:\\MISC\\IU_downloads\\PSP_Jan_Feb_2019_original.csv"),index=None)
df_transac_fee=pd.DataFrame(pd.read_csv("C:\\MISC\\IU_downloads\\PSP_Jan_Feb_2019_transac_fees.csv"),index=None)
#Extracting only failed transactions from both the files & transforming the dataframe 
df_original_fail=df_original[df_original["success"]==0]
df_transac_fee_fail=df_transac_fee.drop(columns=("Success"))
#Merging the two files so that transaction fees are obtained for each failed transaction
pd_merge_fail=pd.merge(df_original_fail, df_transac_fee_fail, on="PSP",how="inner")
#Extracting only successful transactions from both the files & transforming the dataframe 
df_original_success=df_original[df_original["success"]==1]
df_transac_fee_success=df_transac_fee.drop(columns=("Fail"))
#Merging the two files so that transaction fees are obtained for each successful transaction
pd_merge_success=pd.merge(df_original_success, df_transac_fee_success, on="PSP",how="inner")
#Renaming the column to transaction_fees for each success & failure
pd_merge_fail.rename(columns={"Fail":"transaction_fees"},inplace=True)
pd_merge_success.rename(columns={"Success":"transaction_fees"},inplace=True)
#Concatinating & sorting the final file to obtain the required dataset
pd_merge_t_fee=pd.concat([pd_merge_fail,pd_merge_success])
pd_merge_t_fee_sorted=pd_merge_t_fee.sort_values(by=["tmsp"])